## Setup

In [79]:
import numpy as np
import cmath as math

from math import pi, sqrt
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import X, H, I, SWAP, CZ, CPHASE00, CPHASE01, CPHASE10, CPHASE
from pyquil.parameters import Parameter, quil_sin, quil_cos
from pyquil.quilbase import DefGate
from pyquil.quilatom import unpack_qubit, unpack_classical_reg
#from .quilbase import (Measurement, Gate, Wait, Reset, Halt, Nop, ClassicalTrue,
   #                    ClassicalFalse, ClassicalNot, ClassicalAnd, ClassicalOr, ClassicalMove,
    #                   ClassicalExchange)

def _make_gate(name, num_qubits, num_params=0):
    def gate_function(*params):
        params = list(params)
        stray_qubits = []
        if len(params) < num_params:
            raise ValueError(
                "Wrong number of params for {}. {} given, require {}."
                .format(name, len(params), num_params)
            )
        elif len(params) > num_params:
            stray_qubits = params[num_params:]
            params = params[0:num_params]

        def ctor(*qubits):
            qubits = stray_qubits + list(qubits)
            if len(qubits) != num_qubits:
                raise ValueError(
                    "Wrong number of qubits for {}. {} given, require {}."
                    .format(name, len(qubits), num_qubits)
                )
            return Gate(name, params, [unpack_qubit(q) for q in qubits])
        
        if len(stray_qubits) == num_qubits:
            return Gate(name, params, [unpack_qubit(q) for q in stray_qubits])
        else:
            return ctor
        
    return gate_function


## Initialization

In [2]:
qvm = QVMConnection()
gr_prog = Program()

In [83]:
gr_prog = Program().inst(I(2),I(1))
state = qvm.wavefunction(gr_prog)
print(state)

(1+0j)|00>


## Create Superposition

In [84]:
gr_prog.inst([H(2)])
state = qvm.wavefunction(gr_prog)
print(state)

(0.7071067812+0j)|00> + (0.7071067812+0j)|10>


## Apply Controlled Unitary Gate

In [82]:
#theta = Parameter('theta')
theta = 0
gr_prog.inst([CPHASE10(math.exp(2*pi*1j*theta), 2, 1)])
state = qvm.wavefunction(gr_prog)
print(state)

(0.7071067812+0j)|00> + (0.3820514244+0.5950098395j)|10>


## Apply inverse quantum Fourier transform gate

In [60]:
QFT2_Inverse = np.array([[1,1],
                        [1, math.exp(-pi*1j)]]) * (1/sqrt(2))
Program().defgate("QFT2_Inverse", QFT2_Inverse)
QFT2_Inverse

array([[ 0.70710678+0.00000000e+00j,  0.70710678+0.00000000e+00j],
       [ 0.70710678+0.00000000e+00j, -0.70710678-8.65956056e-17j]])

In [61]:
gr_prog.inst(QFT2_Inverse(2))
state = qvm.wavefunction(gr_prog)
print(state)

TypeError: 'numpy.ndarray' object is not callable

In [76]:

phi = 2*pi
gr_prog.inst([CPHASE10(phi,2,1)])
state = qvm.wavefunction(gr_prog)
print(state)

NameError: name 'Gate' is not defined

ValueError: Matrix must be unitary.

In [50]:
theta = Parameter('theta')
crx = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, quil_cos(theta / 2), -1j * quil_sin(theta / 2)], [0, 0, -1j * quil_sin(theta / 2), quil_cos(theta / 2)]])

dg = DefGate('CRX', crx, [theta])
CRX = dg.get_constructor()

p = Program()
p.inst(dg)
p.inst(H(0))
p.inst(CRX(np.pi/2)(0, 1))

wavefunction = qvm.wavefunction(p)
print(wavefunction)

(1-2.4492935982947064e-16j)